In [1]:
###################Encryption, Update and Keystream function definition Start#####################

def keystream(L,N):                 #Keystream bit function
    z = L[30] ^ N[1] ^ N[6] ^ N[15] ^  N[17] ^ N[23] ^ N[28] ^ N[34] ^ (N[4]&L[6] ^ L[8]&L[10] ^ L[32]&L[17] ^ L[19]&L[23] ^ N[4]&L[32]&N[38])
    return z
    
def LFSR_fw_update(L):              #LFSR feedback function in forward direction
    l = (L[0] ^ L[14] ^ L[20] ^ L[34] ^ L[43] ^ L[54])
    return l
    
def LFSR_bk_update(L):              #LFSR feedback function in backward direction
    lb = L[60] ^ L[13] ^ L[19] ^ L[33] ^ L[42] ^ L[53]
    return lb

def NFSR_fw_update(N,l0,key,t):     #NFSR feedback function in forward direction
    n = l0 ^ (((t%80)>>4)&1) ^ key[t%80] ^ N[0] ^ N[13] ^ N[19] ^ N[35] ^ N[39] ^ N[2]&N[25] ^ N[3]&N[5] ^ N[7]&N[8] ^ N[14]&N[21] ^ N[16]&N[18] ^ N[22]&N[24] ^ N[26]&N[32] ^ N[33]&N[36]&N[37]&N[38] ^ N[10]&N[11]&N[12] ^ N[27]&N[30]&N[31]
    return n
    
def NFSR_bk_update(N,l0,key,t):     #NFSR feedback function in backward direction
    nb = l0 ^ (((t%80)>>4)&1) ^ key[t%80] ^ N[39] ^ N[12] ^ N[18] ^ N[34] ^ N[38] ^ N[1]&N[24] ^ N[2]&N[4] ^ N[6]&N[7] ^ N[13]&N[20] ^ N[15]&N[17] ^ N[21]&N[23] ^ N[25]&N[31] ^ N[32]&N[35]&N[36]&N[37] ^ N[9]&N[10]&N[11] ^ N[26]&N[29]&N[30]
    return nb


def Encryption():                                #Encryption function
    import random
    key = [random.randint(0,1) for i in range(80)]
    IV = [random.randint(0,1) for i in range(90)]
    N = [IV[i] for i in range(40)]                                      #NFSR initialisation
    L = [IV[40+i] for i in range(50)] + [1,1,1,1,1,1,1,1,1,0,1]         #LFSR initialisation
    
    #Initialisation Phase
    for i in range(320):
        z = keystream(L,N)                         #keystream bit
        l_new = (z ^ LFSR_fw_update(L))            #LFSR feedback bit
        l0 = L[0]
        L = L[1:60] + [l_new,1]                    #LFSR update
    
        n_new = (z ^ NFSR_fw_update(N,l0,key,i))   #NFSR feedback bit
        N = N[1:40] + [n_new]                      #NFSR update
        
    
    #Pseudorandom (PRGA) phase
    key_stream = 100                               #Targeted round (any random number)
    for i in range(key_stream):

        z = keystream(L,N)

        l_new = LFSR_fw_update(L)
        l0 = L[0]
        L = L[1:61] + [l_new]

        n_new = NFSR_fw_update(N,l0,key,i)
        N = N[1:40] + [n_new]
    
    return L,N,key                                 
        
####################Encryption, Update and Keystream function definition End#####################




#Plantlet Encryption
#Specificaton
#101-bit internal state
#NFSR 40-bit  LFSR 61-bit
#NFSR N       LFSR L
#Key 80-bit
#IV  90-bit


import math
import random
import time
from datetime import datetime


state_len = 101      #state_size
key_len = 80
r = [61, 40]         #Size of the registers LFSR (L) and NFSR (N) respectively
N_round = 250       #Total number of rounds needed
G_pos_L = []   #Guessing position LFSR
G_pos_N = []   #Guessing position NFSR
G_pos_K = []   #Guessing position secret key
sample = 5
z_act = 1
tolerance = 1
Error = [i for i in range(-tolerance,tolerance+1,1)]
guess = len(G_pos_L+G_pos_N+G_pos_K)


bitlen = math.floor(math.log(max(r),2)) + 1   #Number of bit that is needed to represent the max. HD 
#bitlen = math.floor(math.log(state_len,2)) + 1

N_round_f = math.ceil(N_round/2)                #No. of forward rounds
N_round_b = math.floor(N_round/2)               #No. of backward rounds

print("Date and Time ",datetime.now())
print("HD/Reg Model Plantlet")
print("Number of rounds in forward ", N_round_f," and in backward ", N_round_b)
print("Total number of rounds",N_round)
print("The Guessed bits are LFSR ",G_pos_L,"\nNFSR ",G_pos_N, "\nKEY ",G_pos_K)
print("Number of Guessed bit ", guess)
print("Error Tolerance class ", Error)
print("Keystream equation ", z_act)
print("No. of Sample ", sample)

filename = 'Plantlet_HD-Reg_g'+str(guess)+'_z_'+str(z_act)+'_tolerance-'+str(tolerance)+'_fwd+bkd_Rounds'+str(N_round)+'.txt'

file = open(filename,'a')
file.write("\n\n\nDate and Time " + str(datetime.now()))
file.write("\nHD-Reg Model Plantlet")
file.write("\nNumber of rounds in forward " + str(N_round_f) +" and in backward " + str(N_round_b))
file.write("\nTotal number of rounds" + str(N_round))
file.write("\nThe Guessed bits are LFSR " + str(G_pos_L) + "\nNFSR " + str(G_pos_N) + "\n KEY "+ str(G_pos_K))
file.write("\nNumber of Guessed bit "+ str(guess))
file.write("\nError Tolerance class "+ str(Error))
file.close()

cnt_success = 0
runtime = [0]*sample


for loop in range(sample):
    ###############Plantlet cipher with a random key and IV to generate keystream and HD ####################################

    L_org, N_org, K_org = Encryption()         #Internal state picked from the pseudorandom phase
                                               #K_org is the secret key. We need it while updating the state for HD information
        
    L = L_org[:]
    N = N_org[:]

    Z_f = [0]*N_round_f              #Array to store the forward keystream bits
    Z_b = [0]*N_round_b              #Array to store the backward keystream bits

    #Array to store the HD of Internal states in forward directions
    hamm_dist_L_f = [0 for i in range(N_round_f-1)]   
    hamm_dist_N_f = [0 for i in range(N_round_f-1)]

    #Array to store the HD of Internal states in backward directions
    hamm_dist_L_b = [0 for i in range(N_round_b)]
    hamm_dist_N_b = [0 for i in range(N_round_b)]

    #Information on keystream and HD/Reg for internal state in backward direction
    for i in range(N_round_b): 
        l = LFSR_bk_update(L)                    #Feedback function of LFSR
        n = NFSR_bk_update(N,l,K_org,(79-i))     #Feedback function of NFSR 
        temp1 = N + L                            #Internal state before update

        L = [l] + L[:r[0]-1]                     #State update
        N = [n] + N[:r[1]-1] 

        temp2 = N + L                             #Internal state after update

        Z_b[i] = keystream(L,N)            #Storing the keystream bits at each round in backward direction


        #Storing the HD of  internal  states in backward direction
        for l in range(state_len):
            if l < r[1]:
                hamm_dist_N_b[i] += (temp1[l]^temp2[l])   
            else:
                hamm_dist_L_b[i] += (temp1[l]^temp2[l])


    L = L_org[:]
    N = N_org[:]
    #Information on keystream and HD/Reg in forward direction
    for i in range(N_round_f):
        Z_f[i] = keystream(L,N)            #Storing the keystream bits at each round in forward direction
        temp1 = N + L                      #Internal state before update

        if i < N_round_f -1:
            #updating the registers
            l = LFSR_fw_update(L)
            n = NFSR_fw_update(N,L[0],K_org,i)

            L = L[1:r[0]] + [l]
            N = N[1:r[1]] + [n]

            temp2 = N + L                 #Internal state after update
            #Storing the HD of  internal  states in forward direction
            for l in range(state_len):
                if l < r[1]:
                    hamm_dist_N_f[i] += (temp1[l]^temp2[l]) 
                else:
                    hamm_dist_L_f[i] += (temp1[l]^temp2[l])


    #############Injection Error to the Original Hamming Distance ##########################
    cntf = 0
    cntb = 0
    if len(Error) > 1:
        for i in range(N_round_b):
            temp1 = random.choice(Error)
            temp2 = random.choice(Error)
            if temp1 != 0:
                cntb += 1
            if temp2 != 0:
                cntb += 1
                
            hamm_dist_N_b[i] += temp1
            hamm_dist_L_b[i] += temp2
            if hamm_dist_N_b[i] < 0:
                hamm_dist_N_b[i] = 0
            if hamm_dist_N_b[i] > r[1]:
                hamm_dist_N_b[i] = r[1]
                
            if hamm_dist_L_b[i] < 0:
                hamm_dist_L_b[i] = 0
            if hamm_dist_L_b[i] > r[0]:
                hamm_dist_L_b[i] = r[0]
            
            

        for i in range(N_round_f-1):
            temp1 = random.choice(Error)
            temp2 = random.choice(Error)
            if temp1 != 0:
                cntf += 1
            if temp2 != 0:
                cntf += 1
                
            hamm_dist_N_f[i] += temp1
            hamm_dist_L_f[i] += temp2
            if hamm_dist_N_f[i] < 0:
                hamm_dist_N_f[i] = 0
            if hamm_dist_N_f[i] > r[1]:
                hamm_dist_N_f[i] = r[1]
                
            if hamm_dist_L_f[i] < 0:
                hamm_dist_L_f[i] = 0
            if hamm_dist_L_f[i] > r[0]:
                hamm_dist_L_f[i] = r[0]
    print(cntf,cntb)

    ###############SMT Modelling to recover state bits from keystream and HD/Reg#########################################
    from z3 import *
    m = Solver()

    #Defining variables of the format BitVec(.,1)
    L_var_org = [BitVec('l%d'%i,1) for i in range(r[0])]          #For register L
    N_var_org = [BitVec('n%d'%i,1) for i in range(r[1])]          #For register N
    K = [BitVec('k%d'%i,1) for i in range(len(K_org))]

    L_f = [BitVec('L_f%d'%i,1) for i in range(1,N_round_f,1)]       #Dummy variables for LFSR L in forward direction
    N_f = [BitVec('N_f%d'%i,1) for i in range(1,N_round_f,1)]       #Dummy variables for NFSR N in forward direction


    L_b = [BitVec('L_b%d'%i,1) for i in range(1,N_round_b+1,1)]     #Dummy variables for LFSR L in backward direction
    N_b = [BitVec('N_b%d'%i,1) for i in range(1,N_round_b+1,1)]     #Dummy variables for NFSR N in backward direction


    L = L_var_org[:]
    N = N_var_org[:]


    #To check the satisfiability of equations guess all variables otherwise guess as per the requirement
    #L_org, N_org are the original register of the targeted internal state
    #K_org refers to the original Key-bits
    #Guess accordingly 
    for i in G_pos_L:
        m.add(L[i] == L_org[i])
    for i in G_pos_N:
        m.add(N[i] == N_org[i])
    for i in G_pos_K:
        m.add(K[i] == K_org[i])
    #Guess End


    #Array to store the keystream equations    
    Z_f_equ = [0]*N_round_f        
    Z_b_equ = [0]*N_round_b

    #Array to store the dummy variable equations for both register L and B
    L_f_equ = [0]*(N_round_f-1)
    N_f_equ = [0]*(N_round_f-1)

    L_b_equ = [0]*N_round_b
    N_b_equ = [0]*N_round_b

    #Array to store the HD/Reg equations
    hamm_dist_L_f_equ = [0 for i in range(N_round_f-1)]          #For LFSR in forward direction
    hamm_dist_N_f_equ = [0 for i in range(N_round_f-1)]          #For NFSR in forward direction
    hamm_dist_L_b_equ = [0 for i in range(N_round_b)]            #For LFSR in backward direction
    hamm_dist_N_b_equ = [0 for i in range(N_round_b)]            #For NFSR in backward direction

    #Equation generation steps for backward internal states
    for i in range(N_round_b):

        #Dummy Variable equated to the feedback function
        l = LFSR_bk_update(L)
        n = NFSR_bk_update(N,l,K,79-i)

        temp1 = N + L                      #Internal state before update

        L_b_equ[i] = (L_b[i] == l)          
        N_b_equ[i] = (N_b[i] == n)

        #State updated with dummy variables
        L = [L_b[i]] + L[:r[0]-1]
        N = [N_b[i]] + N[:r[1]-1]

        #Hamming distance equation
        temp2 = N + L                      #Internal state after update
        for l in range(state_len):
            if l < r[1]:
                hamm_dist_N_b_equ[i] += ZeroExt(bitlen-1,(temp1[l]^temp2[l]))
            else:
                hamm_dist_L_b_equ[i] += ZeroExt(bitlen-1,(temp1[l]^temp2[l]))
                
        m.add(UGE(hamm_dist_L_b_equ[i],max(hamm_dist_L_b[i]-tolerance,0)))
        m.add(ULE(hamm_dist_L_b_equ[i],min(hamm_dist_L_b[i]+tolerance,r[0])))

        m.add(UGE(hamm_dist_N_b_equ[i],max(hamm_dist_N_b[i]-tolerance,0)))
        m.add(ULE(hamm_dist_N_b_equ[i],min(hamm_dist_N_b[i]+tolerance,r[1])))
              

        #Keystream equation
        Z_b_equ[i] = (keystream(L,N) == Z_b[i])


    L = L_var_org[:]
    N = N_var_org[:]
    #Equations generation steps for forward internal states
    for i in range(N_round_f):
        Z_f_equ[i] = (keystream(L,N) == Z_f[i])             #Keystream equation


        temp1 = N + L                       #Internal state before update

        if i < N_round_f-1:
            l = LFSR_fw_update(L)
            n = NFSR_fw_update(N,L[0],K,i)

            L_f_equ[i] = (L_f[i] == l)            #Dummy variable equated to the feedback function
            N_f_equ[i] = (N_f[i] == n)

            #Internal state updated with dummy variable
            L = L[1:] + [L_f[i]]
            N = N[1:] + [N_f[i]]

            temp2 = N + L                  #Internal state after update
            #Hamming distance equation
            for l in range(state_len):
                if l < r[1]:
                    hamm_dist_N_f_equ[i] += ZeroExt(bitlen-1,(temp1[l]^temp2[l]))
                else:
                    hamm_dist_L_f_equ[i] += ZeroExt(bitlen-1,(temp1[l]^temp2[l]))
            m.add(UGE(hamm_dist_L_f_equ[i],max(hamm_dist_L_f[i]-tolerance,0)))
            m.add(ULE(hamm_dist_L_f_equ[i],min(hamm_dist_L_f[i]+tolerance,r[0])))

            m.add(UGE(hamm_dist_N_f_equ[i],max(hamm_dist_N_f[i]-tolerance,0)))
            m.add(ULE(hamm_dist_N_f_equ[i],min(hamm_dist_N_f[i]+tolerance,r[1])))
            

    # All Keystream and Dummy variable equations
    if z_act == 1:
        Equ = Z_f_equ + Z_b_equ + L_f_equ + N_f_equ + L_b_equ + N_b_equ
    else:
        Equ = L_f_equ + N_f_equ + L_b_equ + N_b_equ

    m.add(Equ)              #Equation feeded to the model m
    print("Start")
    start = time.time()
    temp = m.check()        #Check the satisfiability of the given system of equations
    end = time.time()

    file = open(filename,'a')
    #If Satisfiable
    if temp == sat:         
        print(temp)
        soln = m.model()           #soln contains the output
        print("SMT Time = ", end - start)         #Time taken by solver to solve
        file.write("\n SMT Time = " + str(end-start))
        runtime[loop] = end-start
        #Extracting the value of the internal state variables
        out_L = [0]*(r[0])
        out_N = [0]*(r[1])
        out_K = [0]*(len(K))
        for i in range(r[0]):
            out_L[i] = soln[L_var_org[i]]
        for i in range(r[1]):
            out_N[i] = soln[N_var_org[i]]
        for i in range(len(K)):
            out_K[i] = soln[K[i]]

        #Check whether the original state is same as the output state
        #L_org, N_org are the original registers whereas out_L and out_N are the output of SMT
        #K_org is the original secret key whereas out_K is the output key bit of SMT
        if K_org == out_K:
            print("******************Matched*********************")
            file.write("\n*********************Matched*****************")
            cnt_success += 1
        else:
            print("############################not matched##############################################")
            print(L_org)
            print(N_org)
            print(out_L)
            print(out_N)
            file.write("\n##################################Not Matched#############################")


    #If the solution is Unsatisfiable, check code and equations
    else:
        print("###################################unsat###################################################")
        file.write("\n#################unsat#################")
        print(temp)
        print(L_org)
        print(N_org)
    file.close()

    
print("No. of Sucessful output ",cnt_success,"  out of ",sample)
file = open(filename,'a')
file.write("\n\n No. of correct ouptput ")
file.write(str(cnt_success))
file.write(" out of sample ")
file.write(str(sample))
file.write("\n")
file.write("Runtime = ")
file.write(str(runtime))
import numpy
runtime = numpy.array(runtime)
file.write("\n\n Mean =  ")
file.write(str(runtime.mean()))
file.write("\n Std = ")
file.write(str(runtime.std()))
file.write("\n")
file.close()

Date and Time  2021-12-09 18:18:25.357627
HD/Reg Model Plantlet
Number of rounds in forward  125  and in backward  125
Total number of rounds 250
The Guessed bits are LFSR  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60] 
NFSR  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39] 
KEY  []
Number of Guessed bit  101
Error Tolerance class  [-1, 0, 1]
Keystream equation  1
No. of Sample  5
166 162
Start
sat
SMT Time =  0.2967362403869629
******************Matched*********************
158 166
Start
sat
SMT Time =  0.561608076095581
******************Matched*********************
158 165
Start
sat
SMT Time =  0.28865694999694824
******************Matched*********************
156 156
Start
sat
SMT Time =  0.33782

In [ ]:
2091/(60*60)

In [ ]:
n = 1000
a = 5
print(pow(1-(a/n),n),pow(math.e,-a))